<a href="https://colab.research.google.com/github/sankeawthong/Project-1-Lita-Chatbot/blob/main/%5B20250930%5D%20LR-BiLSTM(MLP)%20Train_all_in_one_20250928_cic_tiny_slice%20(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

train_all_in_one — CIC fixes + tiny benign slice (2025-09-28, v2)

Why this version?
- Ensures the CIC tiny-slice experiment RUNS BY DEFAULT (no flag needed).
- Always writes either the requested output files OR an explicit SKIPPED/ERROR file explaining why.
- Keeps your earlier fixes (A–C essential, E recommended) intact.

Key outputs from tiny-slice mode (when successful):

  • CIC_IoMT__tiny_benign_slice__summary.csv

  • CIC_IoMT__tiny_benign_slice__binary_metrics.json

  • CIC_IoMT__tiny_benign_slice__Calibrated(isotonic)__binary_metrics.json (+ __meta.json)

  • CIC_IoMT__tiny_benign_slice__Calibrated(temperature)__binary_metrics.json (+ __meta.json)

And the automapper audit files:

  • NF_to_CIC__feature_automap.json, NF_to_CIC_common_features.json

  • CIC_to_NF__feature_automap.json, CIC_to_NF_common_features.json

In [1]:
import os, json, argparse, re, difflib, traceback
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler, QuantileTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline as SkPipeline
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
from sklearn.metrics import (accuracy_score, f1_score, classification_report,
                             average_precision_score, roc_auc_score, precision_recall_curve,
                             confusion_matrix, ConfusionMatrixDisplay)
from sklearn.isotonic import IsotonicRegression
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

In [2]:
# -----------------------------
# Config
# -----------------------------

CFG = {
    "paths": {
        "nf_csv": "/content/Dataset_NF-ToN-IoT.csv",
        "cic_train_csv": "/content/CIC_IoMT_2024_WiFi_MQTT_train.csv",
        "cic_test_csv": "/content/CIC_IoMT_2024_WiFi_MQTT_test.csv",
        "outdir": "/mnt/data/iot_ids_refactor/outputs"
    },
    "label_columns": {
        "binary_candidates": ["Label", "label", "Binary", "binary"],
        "multiclass_candidates": ["Class", "class", "Category", "category"],
        "drop_non_feature_if_present": ["Attack", "attack", "Label", "label", "Class", "class"]
    },
    "train": {
        "random_state": 42,
        "test_size": 0.2,
        "use_smote": True,
        "mlp_hidden_units": 64,
        "max_epochs": 25,
        "batch_size": 2048
    },
    "metrics": {"target_drs": [0.90, 0.95]},
    "calibration": {"cic_calib_frac": 0.10},
    "robust": {"eps": [0.05, 0.10], "pgd_steps": 10, "pgd_alpha": 0.02},
    # E) loosened defaults
    "automap": {"similarity_threshold": 0.75, "max_pairs": 256}
}

In [3]:
# --------------------------
# Canonical feature aliases
# --------------------------
CANON = {
    "duration": ["dur", "flow_duration", "duration", "dur_ms", "flowdur", "flow_dur"],
    "tot_fwd_pkts": ["Tot Fwd Pkts", "tot_fwd_pkts", "total_fwd_packets", "fwd_pkts_tot", "fwd_pkts_total", "fwd_packets_total"],
    "tot_bwd_pkts": ["Tot Bwd Pkts", "tot_bwd_pkts", "total_bwd_packets", "bwd_pkts_tot", "bwd_pkts_total", "bwd_packets_total"],
    "totlen_fwd_pkts": ["TotLen Fwd Pkts", "totlen_fwd_pkts", "total_length_of_fwd_packets", "fwd_pkts_len_tot", "fwd_bytes_total", "total_fwd_bytes"],
    "totlen_bwd_pkts": ["TotLen Bwd Pkts", "totlen_bwd_pkts", "total_length_of_bwd_packets", "bwd_pkts_len_tot", "bwd_bytes_total", "total_bwd_bytes"],
    "fwd_pkt_len_mean": ["Fwd Pkt Len Mean", "fwd_pkt_len_mean", "fwd_packet_length_mean", "fwd_pkt_length_mean"],
    "bwd_pkt_len_mean": ["Bwd Pkt Len Mean", "bwd_pkt_len_mean", "bwd_packet_length_mean", "bwd_pkt_length_mean"],
    "fwd_iat_mean": ["Fwd IAT Mean", "fwd_iat_mean", "fwd_interarrival_mean", "fwd_iat_avg"],
    "bwd_iat_mean": ["Bwd IAT Mean", "bwd_iat_mean", "bwd_interarrival_mean", "bwd_iat_avg"],
    "pkt_len_mean": ["Pkt Len Mean", "pkt_len_mean", "packet_length_mean", "pkt_length_mean"],
    "pkt_len_std": ["Pkt Len Std", "pkt_len_std", "packet_length_std", "pkt_length_std"],
    "flow_pkts_s": ["Flow Pkts/s", "flow_pkts_s", "packets_per_second", "pkts_per_sec", "pkt_rate"],
    "flow_byts_s": ["Flow Byts/s", "flow_byts_s", "bytes_per_second", "byte_rate", "bytes_per_sec", "throughput"],
    "protocol": ["Protocol", "proto", "protocol", "protocol_type", "l4_proto"],
    "src_port": ["Src Port", "src_port", "sport", "source_port"],
    "dst_port": ["Dst Port", "dst_port", "dport", "destination_port"],
    "flags": ["flags", "tcp_flags", "flag_count", "tcpflag", "flag_total"]
}

In [4]:
# -----------------
# Feature utilities
# -----------------
def build_rename_map(df_cols):
    lower = {c.lower(): c for c in df_cols}
    rename = {}
    for canon, aliases in CANON.items():
        for a in aliases:
            key = a.lower()
            if key in lower:
                rename[lower[key]] = canon
                break
    return rename

def normalize_features(df):
    rename = build_rename_map(df.columns)
    return df.rename(columns=rename)

In [5]:
# -----------------
# I/O helpers
# -----------------
def detect_label_column(df, candidates):
    for c in candidates:
        if c in df.columns:
            return c
    return None

def pick_features(df, drop_cols):
    num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    feat_cols = [c for c in num_cols if c not in set(drop_cols)]
    return feat_cols

def load_dataset(path):
    df_raw = pd.read_csv(path)
    df = normalize_features(df_raw)
    bin_col = detect_label_column(df, CFG["label_columns"]["binary_candidates"])
    mc_col  = detect_label_column(df, CFG["label_columns"]["multiclass_candidates"])
    feat_cols = pick_features(df, CFG["label_columns"]["drop_non_feature_if_present"])
    return df, feat_cols, bin_col, mc_col

In [6]:
# -----------------
# Labels
# -----------------
def map_label_string_to_family(s: str) -> str:
    st = str(s).lower().strip()
    if "benign" in st or "normal" in st:
        return "Benign"
    return "Attack"

def build_binary_labels(df, bin_col, mc_col):
    if bin_col and bin_col in df.columns:
        try:
            y_bin = df[bin_col].astype(int).values
        except ValueError:
            y_bin = df[bin_col].apply(lambda x: 0 if map_label_string_to_family(str(x)) == "Benign" else 1).values
    elif mc_col and mc_col in df.columns:
        try:
            y_bin = (df[mc_col].astype(int).values != 0).astype(int)
        except ValueError:
            y_bin = df[mc_col].apply(lambda x: 0 if map_label_string_to_family(str(x)) == "Benign" else 1).values
    else:
        raise ValueError("No binary or multiclass label column found.")
    return y_bin

def make_one_hot(y, n_classes):
    out = np.zeros((len(y), n_classes), dtype=int)
    out[np.arange(len(y)), y] = 1
    return out

In [7]:
# -----------------
# Calibration
# -----------------
def sigmoid(x): return 1.0/(1.0+np.exp(-x))

def fit_temperature(z_val, y_val, n_iters=300, lr=0.05):
    T = 1.0
    for _ in range(n_iters):
        p = sigmoid(z_val / T)
        p = np.clip(p, 1e-7, 1-1e-7)
        grad = np.mean((p - y_val) * (-z_val/(T*T)))
        T -= lr * grad
        T = float(np.clip(T, 0.05, 50.0))
    return T

def apply_temperature(z, T):
    return sigmoid(z / T)

def calibrate_scores(scores_cal, y_cal_bin, scores_eval, method="temperature"):
    s_cal = np.clip(scores_cal, 1e-6, 1-1e-6)
    s_eval = np.clip(scores_eval, 1e-6, 1-1e-6)
    if method == "isotonic":
        iso = IsotonicRegression(out_of_bounds="clip")
        iso.fit(s_cal, y_cal_bin.astype(int))
        s_eval_cal = iso.predict(s_eval)
        return s_eval_cal, {"method": "isotonic"}
    else:
        z_cal = np.log(s_cal/(1-s_cal))
        T = fit_temperature(z_cal, y_cal_bin.astype(int), n_iters=300, lr=0.05)
        z_eval = np.log(s_eval/(1-s_eval))
        s_eval_cal = apply_temperature(z_eval, T)
        return s_eval_cal, {"method": "temperature", "T": float(T)}

In [8]:
# -----------------
# Metrics & plots
# -----------------
def fpr_at_dr(y_true, scores, target_dr=0.95, positive_label=1):
    thresholds = np.unique(scores)
    thresholds.sort()
    best_fpr = None
    best_thr = None
    for thr in thresholds:
        y_pred = (scores >= thr).astype(int)
        pos = (y_true == positive_label)
        neg = ~pos
        tp = (pos & (y_pred == 1)).sum()
        fn = (pos & (y_pred == 0)).sum()
        dr = tp / max(tp + fn, 1)
        if dr >= target_dr:
            fp = (neg & (y_pred == 1)).sum()
            fpr = fp / max(neg.sum(), 1)
            if best_fpr is None or fpr < best_fpr:
                best_fpr = fpr
                best_thr = thr
    return best_fpr if best_fpr is not None else np.nan, best_thr

def expected_calibration_error(y_true, probas, n_bins=15):
    confidences = probas.max(axis=1)
    predictions = probas.argmax(axis=1)
    correct = (predictions == y_true).astype(float)
    bins = np.linspace(0.0, 1.0, n_bins+1)
    ece = 0.0
    for i in range(n_bins):
        mask = (confidences > bins[i]) & (confidences <= bins[i+1])
        if mask.sum() == 0:
            continue
        acc = correct[mask].mean()
        conf = confidences[mask].mean()
        ece += (mask.mean()) * abs(acc - conf)
    return ece

def plot_pr_curves(y_onehot, probas, class_names, out_png):
    n_classes = y_onehot.shape[1]
    plt.figure()
    for c in range(n_classes):
        precision, recall, _ = precision_recall_curve(y_onehot[:, c], probas[:, c])
        ap = average_precision_score(y_onehot[:, c], probas[:, c])
        plt.plot(recall, precision, label=f"{class_names[c]} (AP={ap:.3f})")
    plt.xlabel("Recall"); plt.ylabel("Precision"); plt.title("Precision-Recall Curves")
    plt.legend(); plt.tight_layout(); plt.savefig(out_png, dpi=180); plt.close()

def plot_confusion(y_true, y_pred, class_names, out_png, normalize='true'):
    cm = confusion_matrix(y_true, y_pred, labels=range(len(class_names)), normalize=normalize)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
    fig, ax = plt.subplots(); disp.plot(ax=ax, values_format=".2f", cmap=None, colorbar=False)
    plt.title("Confusion Matrix" + (f" (normalized={normalize})" if normalize else ""))
    plt.tight_layout(); plt.savefig(out_png, dpi=180); plt.close()

In [9]:
# -----------------
# Models
# -----------------
def build_scaler(name):
    if name == "standard":
        return StandardScaler()
    elif name == "robust":
        return RobustScaler(with_centering=True, with_scaling=True, quantile_range=(25.0, 75.0))
    elif name == "quantile":
        return QuantileTransformer(output_distribution="normal", subsample=200000, random_state=CFG["train"]["random_state"])
    else:
        return StandardScaler()

def get_lr_pipeline(use_smote, random_state, y_train, scaler_name='standard', C=1.0):
    if len(np.unique(y_train)) < 2:
        print("[WARN] Skipping LR pipeline (single-class training data).")
        return None
    lr = LogisticRegression(C=C, max_iter=2000, solver="lbfgs", class_weight="balanced", n_jobs=-1)
    steps = [("scaler", build_scaler(scaler_name))]
    if use_smote:
        steps.append(("smote", SMOTE(random_state=random_state)))
    steps.append(("lr", lr))
    pipe = ImbPipeline(steps=steps)
    return pipe

def fit_lr_then_mlp(X_train, y_train, X_val, y_val, use_smote, random_state, max_epochs, batch_size, hidden_units, alpha=1e-4, scaler_name='standard', C=1.0, early_stopping=True):
    pipe = get_lr_pipeline(use_smote, random_state, y_train, scaler_name=scaler_name, C=C)
    if pipe is None:
        return None, None
    pipe.fit(X_train, y_train)
    scaler = pipe.named_steps["scaler"]
    lr = pipe.named_steps["lr"]
    try:
        Z_train = lr.decision_function(scaler.transform(X_train))
        Z_val   = lr.decision_function(scaler.transform(X_val))
        if Z_train.ndim == 1:
            Z_train = Z_train.reshape(-1,1)
            Z_val   = Z_val.reshape(-1,1)
    except Exception:
        Z_train = np.log(np.clip(lr.predict_proba(scaler.transform(X_train)), 1e-7, 1-1e-7))
        Z_val   = np.log(np.clip(lr.predict_proba(scaler.transform(X_val)), 1e-7, 1-1e-7))
        if Z_train.ndim == 1:
            Z_train = Z_train.reshape(-1,1)
        if Z_val.ndim == 1:
            Z_val = Z_val.reshape(-1,1)
    mlp = MLPClassifier(hidden_layer_sizes=(hidden_units,), alpha=alpha,
                        batch_size=batch_size, learning_rate_init=1e-3,
                        max_iter=max_epochs, random_state=random_state,
                        early_stopping=early_stopping, n_iter_no_change=5, validation_fraction=0.1)
    mlp.fit(Z_train, y_train)
    return pipe, mlp

In [10]:
# ------------------------------
# Binary / Multiclass evaluation
# ------------------------------
def eval_binary(y_true, prob_pos, tag, outdir):
    if prob_pos is None or len(prob_pos) == 0:
        with open(os.path.join(outdir, f"{tag}__binary_report.txt"), "w") as f:
            f.write("Binary evaluation skipped due to insufficient classes or probabilities.\n")
        return
    auc = roc_auc_score(y_true, prob_pos)
    aupr = average_precision_score(y_true, prob_pos)
    metrics = {"roc_auc": float(auc), "aupr": float(aupr)}
    for dr in CFG["metrics"]["target_drs"]:
        fpr, thr = fpr_at_dr(y_true, prob_pos, target_dr=dr)
        metrics[f"fpr@dr={dr}"] = float(fpr) if fpr==fpr else None
        metrics[f"thr@dr={dr}"] = float(thr) if thr is not None else None
    os.makedirs(outdir, exist_ok=True)
    with open(os.path.join(outdir, f"{tag}__binary_metrics.json"), "w") as f:
        json.dump(metrics, f, indent=2)
    best_thr = metrics.get("thr@dr=0.95", 0.5)
    y_pred = (prob_pos >= (best_thr if best_thr is not None else 0.5)).astype(int)
    report = classification_report(y_true, y_pred, digits=4)
    with open(os.path.join(outdir, f"{tag}__binary_report.txt"), "w") as f:
        f.write(report)

def eval_multiclass(y_true, probas, class_names, tag, outdir, train_classes=None):
    if probas is None or probas.shape[0] == 0:
        with open(os.path.join(outdir, f"{tag}__multiclass_metrics.json"), "w") as f:
            json.dump({"accuracy": np.nan, "f1_macro": np.nan,
                       "aupr_micro": np.nan, "aupr_per_class": {}, "ece": np.nan}, f, indent=2)
        return
    test_classes = sorted(np.unique(y_true))
    if train_classes is None:
        k = min(len(test_classes), probas.shape[1])
        probas_aligned = np.zeros((probas.shape[0], len(test_classes)))
        probas_aligned[:, :k] = probas[:, :k]
        used_classes = test_classes
    else:
        used_classes = test_classes
        probas_aligned = np.zeros((probas.shape[0], len(test_classes)))
        for j, cls in enumerate(train_classes):
            if cls in test_classes:
                idx = test_classes.index(cls)
                probas_aligned[:, idx] = probas[:, j]
    y_pred = probas_aligned.argmax(axis=1)
    acc = accuracy_score(y_true, y_pred)
    f1m = f1_score(y_true, y_pred, average="macro")
    class_names_eval = [f"C{c}" for c in used_classes]
    y_idx = np.array([used_classes.index(c) for c in y_true])
    y_onehot = make_one_hot(y_idx, len(used_classes))
    aupr_micro = average_precision_score(y_onehot, probas_aligned, average="micro")
    aupr_per = {class_names_eval[i]: float(average_precision_score(y_onehot[:, i], probas_aligned[:, i]))
                for i in range(len(used_classes))}
    ece = expected_calibration_error(y_idx, probas_aligned, n_bins=15)
    with open(os.path.join(outdir, f"{tag}__multiclass_metrics.json"), "w") as f:
        json.dump({"accuracy": float(acc), "f1_macro": float(f1m),
                   "aupr_micro": float(aupr_micro),
                   "aupr_per_class": aupr_per, "ece": float(ece)}, f, indent=2)
    plot_pr_curves(y_onehot, probas_aligned, class_names_eval, os.path.join(outdir, f"{tag}__pr_curves.png"))
    plot_confusion(y_idx, y_pred, class_names_eval, os.path.join(outdir, f"{tag}__confusion.png"))

In [11]:
# --------------------------------------------------------
# Adversarial utils (feature-space on standardized inputs)
# --------------------------------------------------------
def fgsm(X_std, y, model, eps):
    W = model.coef_.reshape(1, -1)
    z = model.decision_function(X_std)
    p = 1.0/(1.0+np.exp(-z))
    grad = (p - y.reshape(-1))[:, None] * W
    return X_std + eps * np.sign(grad)

def pgd(X_std, y, model, eps, alpha, steps):
    X_adv = X_std.copy()
    for _ in range(steps):
        X_adv = fgsm(X_adv, y, model, alpha)
        delta = np.clip(X_adv - X_std, -eps, eps)
        X_adv = X_std + delta
    return X_adv

def clip_to_train_range(X_std, scaler, train_min, train_max):
    if hasattr(scaler, 'inverse_transform'):
        X_raw = scaler.inverse_transform(X_std)
    else:
        X_raw = X_std * scaler.scale_ + scaler.mean_
    X_raw = np.clip(X_raw, train_min, train_max)
    return scaler.transform(X_raw)

def augment_with_adversarial(X_tr, y_tr, pipe, lr, frac=0.3, eps=0.05):
    if eps <= 0.0 or frac <= 0.0:
        return X_tr, y_tr
    n = len(y_tr)
    m = max(1, int(frac * n))
    rs = np.random.RandomState(CFG["train"]["random_state"])
    sel = rs.choice(n, size=m, replace=False)
    scaler = pipe.named_steps["scaler"]
    X_std = scaler.transform(X_tr)
    z = lr.decision_function(X_std)
    p = 1.0/(1.0+np.exp(-z))
    W = lr.coef_.reshape(1, -1)
    grad = (p - y_tr.reshape(-1))[:, None] * W
    X_std_adv = X_std.copy()
    X_std_adv[sel] = X_std_adv[sel] + eps * np.sign(grad[sel])
    X_std_adv = clip_to_train_range(X_std_adv, scaler, X_tr.min(axis=0), X_tr.max(axis=0))
    X_adv = scaler.inverse_transform(X_std_adv) if hasattr(scaler, "inverse_transform") else (X_std_adv * scaler.scale_ + scaler.mean_)
    X_mix = X_tr.copy()
    X_mix[sel] = X_adv[sel]
    return X_mix, y_tr

In [12]:
# -----------------
# Automapper
# -----------------
_token_map = {
    "packets":"pkts","packet":"pkt","length":"len","average":"mean","stddev":"std","std_dev":"std",
    "backward":"bwd","forward":"fwd","persec":"persec","per_second":"persec","rate":"persec",
    "bytes":"byts","byte":"byt","duration":"dur","interarrival":"iat","src":"src","dst":"dst"
}
def keyify(name: str) -> str:
    s = re.sub(r'[^a-zA-Z0-9]+', '', name.lower())
    for k,v in _token_map.items():
        s = s.replace(k, v)
    return s

def automap_features(dfA, featsA, dfB, featsB, threshold=0.86, max_pairs=64):
    keysA = {f: keyify(f) for f in featsA}
    keysB = {f: keyify(f) for f in featsB}
    pairs = []
    for a,ka in keysA.items():
        best_b = None; best_sim = 0.0
        for b,kb in keysB.items():
            sim = difflib.SequenceMatcher(None, ka, kb).ratio()
            if sim > best_sim:
                best_sim, best_b = sim, b
        if best_sim >= threshold and np.issubdtype(dfA[a].dtype, np.number) and np.issubdtype(dfB[best_b].dtype, np.number):
            pairs.append((a, best_b, float(best_sim)))
    pairs.sort(key=lambda x: -x[2])
    usedA, usedB, final = set(), set(), []
    for a,b,sim in pairs:
        if a in usedA or b in usedB: continue
        final.append((a,b,sim))
        usedA.add(a); usedB.add(b)
        if len(final) >= max_pairs: break
    return final

def apply_automap_and_rename(df_src, feats_src, df_tgt, feats_tgt, outdir, tag_prefix):
    common = list(sorted(set(feats_src).intersection(set(feats_tgt))))
    audit = {"mode": "intersection", "count": len(common), "pairs": []}
    audit_path = os.path.join(outdir, f"{tag_prefix}__feature_automap.json")
    if len(common) > 0:
        with open(audit_path, "w") as f:
            json.dump(audit, f, indent=2)
        with open(os.path.join(outdir, f"{tag_prefix}_common_features.json"), "w") as f:
            json.dump({"count": len(common), "features": common, "audit": os.path.basename(audit_path)}, f, indent=2)
        return df_src, df_tgt, common, audit_path
    matches = automap_features(df_src, feats_src, df_tgt, feats_tgt,
                               threshold=CFG["automap"]["similarity_threshold"],
                               max_pairs=CFG["automap"]["max_pairs"])
    audit = {"mode": "automap", "count": len(matches),
             "pairs": [{"src":a,"tgt":b,"similarity":sim} for a,b,sim in matches]}
    df_tgt2 = df_tgt.copy()
    rename_map = {b:a for a,b,_ in matches}
    df_tgt2 = df_tgt2.rename(columns=rename_map)
    common2 = [a for a,_,_ in matches]
    audit["renamed"] = rename_map
    with open(audit_path, "w") as f:
        json.dump(audit, f, indent=2)
    with open(os.path.join(outdir, f"{tag_prefix}_common_features.json"), "w") as f:
        json.dump({"count": len(common2), "features": common2, "audit": os.path.basename(audit_path)}, f, indent=2)
    return df_src, df_tgt2, common2, audit_path

In [13]:
# -----------------
# NF in-domain
# -----------------
def run_in_domain_nf(nf_df, nf_feats, nf_bin_col, nf_mc_col, outdir):
    yb = build_binary_labels(nf_df, nf_bin_col, nf_mc_col)
    X = nf_df[nf_feats].values
    X_tr, X_te, y_tr, y_te = train_test_split(X, yb, test_size=CFG["train"]["test_size"],
                                              stratify=yb, random_state=CFG["train"]["random_state"])
    pipe, mlp = fit_lr_then_mlp(X_tr, y_tr, X_te, y_te,
                                CFG["train"]["use_smote"], CFG["train"]["random_state"],
                                CFG["train"]["max_epochs"], CFG["train"]["batch_size"],
                                CFG["train"]["mlp_hidden_units"])
    prob_pos = None; lr=None; scaler=None
    if pipe is not None:
        scaler = pipe.named_steps["scaler"]; lr = pipe.named_steps["lr"]
        try:
            Z_te = lr.decision_function(scaler.transform(X_te))
        except Exception:
            Z_te = np.log(np.clip(lr.predict_proba(scaler.transform(X_te)), 1e-7, 1-1e-7))
        if Z_te.ndim == 1: Z_te = Z_te.reshape(-1,1)
        prob_pos = mlp.predict_proba(Z_te)[:,1] if mlp is not None else lr.predict_proba(scaler.transform(X_te))[:,1]
    eval_binary(y_te, prob_pos, "NF_ToN_IoT__in_domain", outdir)

    if pipe is not None:
        Xte_std = scaler.transform(X_te)
        for eps in CFG["robust"]["eps"]:
            X_fgsm = fgsm(Xte_std, y_te, lr, eps)
            X_fgsm = clip_to_train_range(X_fgsm, scaler, X_tr.min(axis=0), X_tr.max(axis=0))
            z_fgsm = lr.decision_function(X_fgsm)
            prob_fgsm = 1.0/(1.0+np.exp(-z_fgsm))
            eval_binary(y_te, prob_fgsm, f"NF_ToN_IoT__in_domain__FGSM_eps={eps}", outdir)

            X_pgd = pgd(Xte_std, y_te, lr, eps, CFG["robust"]["pgd_alpha"], CFG["robust"]["pgd_steps"])
            X_pgd = clip_to_train_range(X_pgd, scaler, X_tr.min(axis=0), X_tr.max(axis=0))
            z_pgd = lr.decision_function(X_pgd)
            prob_pgd = 1.0/(1.0+np.exp(-z_pgd))
            eval_binary(y_te, prob_pgd, f"NF_ToN_IoT__in_domain__PGD_eps={eps}", outdir)

    if nf_mc_col:
        ym = nf_df[nf_mc_col].astype(int).values
        Xm_tr, Xm_te, ym_tr, ym_te = train_test_split(X, ym, test_size=CFG["train"]["test_size"],
                                                      stratify=ym, random_state=CFG["train"]["random_state"])
        if len(np.unique(ym_tr)) >= 2:
            pipe_m = SkPipeline([("scaler", StandardScaler()),
                                 ("clf", LogisticRegression(max_iter=2000, n_jobs=-1))])
            pipe_m.fit(Xm_tr, ym_tr)
            probas = pipe_m.predict_proba(Xm_te)
            class_names = [f"C{c}" for c in sorted(np.unique(ym))]
            eval_multiclass(ym_te, probas, class_names, "NF_ToN_IoT__in_domain_native_mc", outdir, train_classes=list(pipe_m.named_steps["clf"].classes_))

In [14]:
# -----------------------------------------------------
# CIC in-domain (train→test) collapsed-binary (guarded)
# -----------------------------------------------------
def run_in_domain_cic_collapsed(cic_tr_df, cic_tr_feats, cic_tr_mc_col,
                                cic_te_df, cic_te_feats, cic_te_mc_col, outdir, calib_method="temperature"):
    common = list(sorted(set(cic_tr_feats).intersection(set(cic_te_feats))))
    if len(common) == 0:
        with open(os.path.join(outdir, "CIC_IoMT__train_to_test__binary_from_mc__SKIPPED.txt"), "w") as f:
            f.write("Skipped: no shared numeric features between CIC_train and CIC_test.\n")
        return

    Xtr = cic_tr_df[common].values
    Xte_full = cic_te_df[common].values
    ym_tr = cic_tr_df[cic_tr_mc_col].astype(int).values if cic_tr_mc_col else None
    ym_te_full = cic_te_df[cic_te_mc_col].astype(int).values if cic_te_mc_col else None
    if ym_tr is None or ym_te_full is None or len(np.unique(ym_tr)) < 2:
        with open(os.path.join(outdir, "CIC_IoMT__train_to_test__binary_from_mc__SKIPPED.txt"), "w") as f:
            f.write("Skipped: multiclass labels missing or single-class in CIC_train/CIC_test.\n")
        return

    pipe_m = SkPipeline([("scaler", StandardScaler()),
                         ("clf", LogisticRegression(max_iter=2000, n_jobs=-1))])
    pipe_m.fit(Xtr, ym_tr)
    X_cal, X_eval, y_cal, y_eval = train_test_split(Xte_full, ym_te_full, test_size=1.0-CFG["calibration"]["cic_calib_frac"],
                                                    stratify=ym_te_full, random_state=CFG["train"]["random_state"])
    probas_cal = pipe_m.predict_proba(X_cal)
    probas_eval = pipe_m.predict_proba(X_eval)

    ben_mask_cal = (y_cal == 0)
    if ben_mask_cal.sum() == 0:
        with open(os.path.join(outdir, "CIC_IoMT__train_to_test__binary_from_mc__SKIPPED.txt"), "w") as f:
            f.write("Skipped: no benign in CIC_test calibration split; cannot align benign column.\n")
        return

    mean_probs_on_ben = probas_cal[ben_mask_cal].mean(axis=0)
    benign_idx_aligned = int(np.argmax(mean_probs_on_ben))
    if mean_probs_on_ben[benign_idx_aligned] < 0.25:
        with open(os.path.join(outdir, "CIC_IoMT__train_to_test__binary_from_mc__SKIPPED.txt"), "w") as f:
            f.write(f"Skipped: aligned 'benign' column probability on benign-cal < 0.25 ({mean_probs_on_ben[benign_idx_aligned]:.3f}).\n")
        return

    s_attack_cal  = 1.0 - np.clip(probas_cal[:,  benign_idx_aligned], 1e-6, 1-1e-6)
    s_attack_eval = 1.0 - np.clip(probas_eval[:, benign_idx_aligned], 1e-6, 1-1e-6)

    auc_cal = roc_auc_score((y_cal != 0).astype(int), s_attack_cal)
    if auc_cal < 0.5:
        s_attack_cal  = 1.0 - s_attack_cal
        s_attack_eval = 1.0 - s_attack_eval

    eval_binary((y_eval != 0).astype(int), s_attack_eval,
                "CIC_IoMT__train_to_test__binary_from_mc", outdir)

    s_eval_cal, meta = calibrate_scores(s_attack_cal, (y_cal != 0).astype(int),
                                        s_attack_eval, method=calib_method)
    eval_binary((y_eval != 0).astype(int), s_eval_cal,
                f"CIC_IoMT__train_to_test__binary_from_mc__Calibrated({calib_method})", outdir)
    with open(os.path.join(outdir, f"CIC_IoMT__train_to_test__binary_from_mc__Calibrated({calib_method})__meta.json"), "w") as f:
        json.dump({"benign_alignment": {"column_index": benign_idx_aligned,
                                        "mean_prob_on_benign_cal": float(mean_probs_on_ben[benign_idx_aligned])},
                   **meta}, f, indent=2)

In [15]:
# ------------------------------
# CIC native multiclass (always)
# ------------------------------
def run_in_domain_cic_native_mc(cic_tr_df, cic_tr_feats, cic_tr_mc_col,
                                cic_te_df, cic_te_feats, cic_te_mc_col, outdir):
    common = list(sorted(set(cic_tr_feats).intersection(set(cic_te_feats))))
    if len(common) == 0:
        return
    Xtr = cic_tr_df[common].values
    Xte = cic_te_df[common].values
    ym_tr = cic_tr_df[cic_tr_mc_col].astype(int).values if cic_tr_mc_col else None
    ym_te = cic_te_df[cic_te_mc_col].astype(int).values if cic_te_mc_col else None
    if ym_tr is None or ym_te is None or len(np.unique(ym_tr)) < 2:
        return
    pipe_m = SkPipeline([("scaler", StandardScaler()),
                         ("clf", LogisticRegression(max_iter=2000, n_jobs=-1))])
    pipe_m.fit(Xtr, ym_tr)
    probas = pipe_m.predict_proba(Xte)
    class_names = [f"C{c}" for c in sorted(np.unique(ym_te))]
    train_classes = list(pipe_m.named_steps["clf"].classes_)
    # Use eval_multiclass to write metrics + plots
    eval_multiclass(ym_te, probas, class_names, "CIC_IoMT__native_multiclass__train_to_test", outdir, train_classes=train_classes)

In [16]:
# ----------------------------------------------------------------------
# CIC tiny benign slice experiment (writes explicit SKIPPED/ERROR files)
# ----------------------------------------------------------------------
def run_cic_with_tiny_benign_slice(cic_tr_df, cic_tr_feats, cic_tr_bin_col, cic_tr_mc_col,
                                   cic_te_df, cic_te_feats, cic_te_bin_col, cic_te_mc_col,
                                   outdir, slice_frac=0.015, seed=42):
    os.makedirs(outdir, exist_ok=True)
    try:
        rs = np.random.RandomState(seed)

        # 0) Feature intersection
        common = list(sorted(set(cic_tr_feats).intersection(set(cic_te_feats))))
        if len(common) == 0:
            with open(os.path.join(outdir, "CIC_IoMT__tiny_benign_slice__SKIPPED.txt"), "w") as f:
                f.write("Skipped: no shared numeric features between CIC_train and CIC_test.\n")
            return

        # 1) Labels
        if cic_tr_mc_col is None or cic_te_mc_col is None:
            with open(os.path.join(outdir, "CIC_IoMT__tiny_benign_slice__SKIPPED.txt"), "w") as f:
                f.write("Skipped: multiclass label column not found in CIC train/test.\n")
            return
        y_tr_mc = cic_tr_df[cic_tr_mc_col].astype(int).values
        X_tr_all = cic_tr_df[common].values
        A_idx = np.where(y_tr_mc != 0)[0]
        if len(A_idx) == 0:
            with open(os.path.join(outdir, "CIC_IoMT__tiny_benign_slice__SKIPPED.txt"), "w") as f:
                f.write("Skipped: CIC_train has no attacks; cannot build binary head.\n")
            return
        X_attack_all = X_tr_all[A_idx]

        y_te_mc = cic_te_df[cic_te_mc_col].astype(int).values
        X_te_all = cic_te_df[common].values
        ben_idx_all = np.where(y_te_mc == 0)[0]
        atk_idx_all = np.where(y_te_mc != 0)[0]
        if len(ben_idx_all) == 0:
            with open(os.path.join(outdir, "CIC_IoMT__tiny_benign_slice__SKIPPED.txt"), "w") as f:
                f.write("Skipped: no benign samples in CIC_test.\n")
            return

        # 2) Sample tiny benign slice
        n_ben_slice = max(1, int(len(ben_idx_all) * slice_frac))
        ben_slice = rs.choice(ben_idx_all, size=n_ben_slice, replace=False)

        # Split benign slice into train/cal halves
        n_ben_cal  = max(1, n_ben_slice // 2)
        n_ben_train = n_ben_slice - n_ben_cal
        rs.shuffle(ben_slice)
        ben_train_idx = ben_slice[:n_ben_train]
        ben_cal_idx   = ben_slice[n_ben_train:]

        # Cal set: include a small set of attacks to balance calibration
        n_atk_cal = min(len(atk_idx_all), n_ben_cal)
        atk_cal_idx = rs.choice(atk_idx_all, size=n_atk_cal, replace=False)

        # Held-out eval: everything except the slice+atk_cal
        held_out_mask = np.ones(len(y_te_mc), dtype=bool)
        held_out_mask[ben_slice] = False
        held_out_mask[atk_cal_idx] = False
        X_eval = X_te_all[held_out_mask]
        y_eval_bin = (y_te_mc[held_out_mask] != 0).astype(int)
        if X_eval.shape[0] == 0 or len(np.unique(y_eval_bin)) < 2:
            with open(os.path.join(outdir, "CIC_IoMT__tiny_benign_slice__SKIPPED.txt"), "w") as f:
                f.write("Skipped: empty or single-class held-out evaluation set after slicing.\n")
            return

        # Train set: CIC_train attacks + tiny benign_train
        X_train = np.vstack([X_attack_all, X_te_all[ben_train_idx]])
        y_train_bin = np.concatenate([np.ones(len(X_attack_all), dtype=int),
                                      np.zeros(len(ben_train_idx), dtype=int)])

        # Calibration set
        X_cal = np.vstack([X_te_all[atk_cal_idx], X_te_all[ben_cal_idx]])
        y_cal_bin = np.concatenate([np.ones(len(atk_cal_idx), dtype=int),
                                    np.zeros(len(ben_cal_idx), dtype=int)])

        # 3) Fit binary LR→MLP head
        pipe_b, mlp_b = fit_lr_then_mlp(
            X_train, y_train_bin,
            X_eval,  y_eval_bin,
            CFG["train"]["use_smote"], CFG["train"]["random_state"],
            CFG["train"]["max_epochs"], CFG["train"]["batch_size"],
            CFG["train"]["mlp_hidden_units"]
        )
        if pipe_b is None:
            with open(os.path.join(outdir, "CIC_IoMT__tiny_benign_slice__SKIPPED.txt"), "w") as f:
                f.write("Skipped: training pipeline failed (single-class or pipeline error).\n")
            return

        scaler = pipe_b.named_steps["scaler"]; lr = pipe_b.named_steps["lr"]
        try:
            Z_eval = lr.decision_function(scaler.transform(X_eval))
            Z_cal  = lr.decision_function(scaler.transform(X_cal))
            if Z_eval.ndim == 1: Z_eval = Z_eval.reshape(-1,1)
            if Z_cal.ndim  == 1: Z_cal  = Z_cal.reshape(-1,1)
            if mlp_b is not None:
                prob_eval = mlp_b.predict_proba(Z_eval)[:,1]
                prob_cal  = mlp_b.predict_proba(Z_cal)[:,1]
            else:
                prob_eval = 1.0/(1.0+np.exp(-Z_eval)).ravel()
                prob_cal  = 1.0/(1.0+np.exp(-Z_cal)).ravel()
        except Exception:
            prob_eval = lr.predict_proba(scaler.transform(X_eval))[:,1]
            prob_cal  = lr.predict_proba(scaler.transform(X_cal))[:,1]

        # 4) Uncalibrated metrics
        auc_unc = float(roc_auc_score(y_eval_bin, prob_eval))
        aupr_unc = float(average_precision_score(y_eval_bin, prob_eval))
        fpr90_unc, thr90_unc = fpr_at_dr(y_eval_bin, prob_eval, target_dr=0.90)
        fpr95_unc, thr95_unc = fpr_at_dr(y_eval_bin, prob_eval, target_dr=0.95)
        eval_binary(y_eval_bin, prob_eval, "CIC_IoMT__tiny_benign_slice", outdir)

        # 5) Calibrated metrics (temperature & isotonic)
        rows = []
        def _add_row(kind, auc, aupr, fpr90, thr90, fpr95, thr95):
            rows.append({"scenario": f"CIC_tiny_slice__{kind}",
                         "roc_auc": auc, "aupr": aupr,
                         "fpr@dr=0.90": fpr90, "thr@dr=0.90": thr90,
                         "fpr@dr=0.95": fpr95, "thr@dr=0.95": thr95})

        _add_row("uncalibrated", auc_unc, aupr_unc, fpr90_unc, thr90_unc, fpr95_unc, thr95_unc)

        for method in ["temperature", "isotonic"]:
            prob_eval_cal, meta = calibrate_scores(prob_cal, y_cal_bin, prob_eval, method=method)
            auc_c = float(roc_auc_score(y_eval_bin, prob_eval_cal))
            aupr_c = float(average_precision_score(y_eval_bin, prob_eval_cal))
            fpr90_c, thr90_c = fpr_at_dr(y_eval_bin, prob_eval_cal, target_dr=0.90)
            fpr95_c, thr95_c = fpr_at_dr(y_eval_bin, prob_eval_cal, target_dr=0.95)
            eval_binary(y_eval_bin, prob_eval_cal,
                        f"CIC_IoMT__tiny_benign_slice__Calibrated({method})", outdir)
            with open(os.path.join(outdir, f"CIC_IoMT__tiny_benign_slice__Calibrated({method})__meta.json"), "w") as f:
                json.dump(meta, f, indent=2)
            _add_row(f"Calibrated({method})", auc_c, aupr_c, fpr90_c, thr90_c, fpr95_c, thr95_c)

        # 6) CSV mini-table + meta
        pd.DataFrame(rows).to_csv(os.path.join(outdir, "CIC_IoMT__tiny_benign_slice__summary.csv"), index=False)

        meta_all = {
            "slice_frac": float(slice_frac),
            "seed": int(seed),
            "n_ben_train": int(len(ben_train_idx)),
            "n_ben_cal": int(len(ben_cal_idx)),
            "n_atk_cal": int(len(atk_cal_idx)),
            "n_eval_total": int(len(y_eval_bin)),
            "class_balance_eval": {"benign": int((y_eval_bin==0).sum()), "attack": int((y_eval_bin==1).sum())},
            "features_used": common
        }
        with open(os.path.join(outdir, "CIC_IoMT__tiny_benign_slice__meta.json"), "w") as f:
            json.dump(meta_all, f, indent=2)

        print("[OK] CIC tiny-benign-slice run complete.")
    except Exception as e:
        with open(os.path.join(outdir, "CIC_IoMT__tiny_benign_slice__ERROR.txt"), "w") as f:
            f.write("Exception occurred during tiny-slice run:\n")
            f.write(str(e) + "\n\n" + traceback.format_exc())

In [17]:
# -----------------
# Cross-domain
# -----------------
def run_cross_domain(nf_df, nf_feats, nf_bin_col, nf_mc_col,
                     cic_tr_df, cic_tr_feats, cic_tr_bin_col, cic_tr_mc_col,
                     cic_te_df, cic_te_feats, cic_te_bin_col, cic_te_mc_col,
                     outdir, automap_min=5):

    # NF → CIC_test
    nf_df2, cic_te_df2, common_nf_cic, audit1 = apply_automap_and_rename(nf_df, nf_feats, cic_te_df, cic_te_feats, outdir, "NF_to_CIC")
    if len(common_nf_cic) < automap_min:
        msg = f"Skipped: shared features {len(common_nf_cic)} < {automap_min}\n"
        with open(os.path.join(outdir, "NF_to_CIC__xfer__SKIPPED.txt"), "w") as f:
            f.write(msg)
    else:
        Xs = nf_df2[common_nf_cic].values
        Xt = cic_te_df2[common_nf_cic].values
        ys = build_binary_labels(nf_df2, nf_bin_col, nf_mc_col)
        yt = build_binary_labels(cic_te_df2, cic_te_bin_col, cic_te_mc_col)
        pipe, mlp = fit_lr_then_mlp(Xs, ys, Xt, np.zeros(len(Xt)),
                                    CFG["train"]["use_smote"], CFG["train"]["random_state"],
                                    CFG["train"]["max_epochs"], CFG["train"]["batch_size"],
                                    CFG["train"]["mlp_hidden_units"])
        prob_pos = None; scaler=None; lr=None
        if pipe is not None:
            scaler = pipe.named_steps["scaler"]; lr = pipe.named_steps["lr"]
            try:
                Zt = lr.decision_function(scaler.transform(Xt))
            except Exception:
                Zt = np.log(np.clip(lr.predict_proba(scaler.transform(Xt)), 1e-7, 1-1e-7))
            if Zt.ndim == 1: Zt = Zt.reshape(-1,1)
            prob_pos = mlp.predict_proba(Zt)[:,1] if mlp is not None else lr.predict_proba(scaler.transform(Xt))[:,1]
        eval_binary(yt, prob_pos, "NF_ToN_IoT__to__CIC_IoMT_test__binary_xfer", outdir)

        if pipe is not None:
            Xt_std = scaler.transform(Xt)
            for eps in CFG["robust"]["eps"]:
                Xt_fgsm = fgsm(Xt_std, yt, lr, eps)
                Xt_fgsm = clip_to_train_range(Xt_fgsm, scaler, Xs.min(axis=0), Xs.max(axis=0))
                z_fgsm = lr.decision_function(Xt_fgsm); prob_fgsm = 1.0/(1.0+np.exp(-z_fgsm))
                eval_binary(yt, prob_fgsm, f"NF_ToN_IoT__to__CIC_IoMT_test__FGSM_eps={eps}", outdir)

                Xt_pgd = pgd(Xt_std, yt, lr, eps, CFG["robust"]["pgd_alpha"], CFG["robust"]["pgd_steps"])
                Xt_pgd = clip_to_train_range(Xt_pgd, scaler, Xs.min(axis=0), Xs.max(axis=0))
                z_pgd = lr.decision_function(Xt_pgd); prob_pgd = 1.0/(1.0+np.exp(-z_pgd))
                eval_binary(yt, prob_pgd, f"NF_ToN_IoT__to__CIC_IoMT_test__PGD_eps={eps}", outdir)

    # CIC_train → NF (guarded)
    cic_tr_df2, nf_df2b, common_cic_nf, audit2 = apply_automap_and_rename(cic_tr_df, cic_tr_feats, nf_df, nf_feats, outdir, "CIC_to_NF")
    if len(common_cic_nf) < automap_min:
        msg = f"Skipped: shared features {len(common_cic_nf)} < {automap_min}\n"
        with open(os.path.join(outdir, "CIC_to_NF__xfer__SKIPPED.txt"), "w") as f:
            f.write(msg)
    else:
        Xs2 = cic_tr_df2[common_cic_nf].values
        Xt2 = nf_df2b[common_cic_nf].values
        ys2 = build_binary_labels(cic_tr_df2, cic_tr_bin_col, cic_tr_mc_col)
        yt2 = build_binary_labels(nf_df2b, nf_bin_col, nf_mc_col)
        pipe2, mlp2 = fit_lr_then_mlp(Xs2, ys2, Xt2, np.zeros(len(Xt2)),
                                      CFG["train"]["use_smote"], CFG["train"]["random_state"],
                                      CFG["train"]["max_epochs"], CFG["train"]["batch_size"],
                                      CFG["train"]["mlp_hidden_units"])
        prob_pos2=None; scaler2=None; lr2=None
        if pipe2 is not None:
            scaler2 = pipe2.named_steps["scaler"]; lr2 = pipe2.named_steps["lr"]
            try:
                Zt2 = lr2.decision_function(scaler2.transform(Xt2))
            except Exception:
                Zt2 = np.log(np.clip(lr2.predict_proba(scaler2.transform(Xt2)), 1e-7, 1-1e-7))
            if Zt2.ndim == 1: Zt2 = Zt2.reshape(-1,1)
            prob_pos2 = mlp2.predict_proba(Zt2)[:,1] if mlp2 is not None else lr2.predict_proba(scaler2.transform(Xt2))[:,1]
        eval_binary(yt2, prob_pos2, "CIC_IoMT_train__to__NF_ToN_IoT__binary_xfer", outdir)

In [18]:
# -----------------
# Main
# -----------------
def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("--grid", action="store_true", help="Run compact NF grid and write grid_summary.json")
    parser.add_argument("--adv-train-eps", type=float, default=0.0)
    parser.add_argument("--adv-train-frac", type=float, default=0.3)
    parser.add_argument("--cic-calib", type=str, choices=["temperature","isotonic"], default="temperature")
    parser.add_argument("--automap-min", type=int, default=5, help="Minimum shared features for cross-domain metrics")
    # Tiny-slice controls
    parser.add_argument("--no-cic-tiny-slice", action="store_true",
                        help="Disable the CIC tiny benign slice experiment (enabled by default).")
    parser.add_argument("--cic-slice-frac", type=float, default=0.015,
                        help="Fraction of CIC_test benign to use for training+calibration (default 0.015 = 1.5%).")
    parser.add_argument("--cic-slice-seed", type=int, default=42,
                        help="Random seed for benign-slice sampling.")
    args, _ = parser.parse_known_args()

    outdir = CFG["paths"]["outdir"]
    os.makedirs(outdir, exist_ok=True)

    # Load datasets
    nf_df, nf_feats, nf_bin_col, nf_mc_col = load_dataset(CFG["paths"]["nf_csv"])
    cic_tr_df, cic_tr_feats, cic_tr_bin_col, cic_tr_mc_col = load_dataset(CFG["paths"]["cic_train_csv"])
    cic_te_df, cic_te_feats, cic_te_bin_col, cic_te_mc_col = load_dataset(CFG["paths"]["cic_test_csv"])

    with open(os.path.join(outdir, "hyperparams.json"), "w") as f:
        json.dump({
            "random_state": CFG["train"]["random_state"],
            "use_smote": CFG["train"]["use_smote"],
            "mlp_hidden_units": CFG["train"]["mlp_hidden_units"],
            "max_epochs": CFG["train"]["max_epochs"],
            "batch_size": CFG["train"]["batch_size"],
            "target_drs": CFG["metrics"]["target_drs"],
            "robust_eps": CFG["robust"]["eps"],
            "pgd_steps": CFG["robust"]["pgd_steps"],
            "pgd_alpha": CFG["robust"]["pgd_alpha"],
            "cic_calib_frac": CFG["calibration"]["cic_calib_frac"],
            "cic_calib_method": args.cic_calib,
            "automap_min": args.automap_min,
            "automap_threshold": CFG["automap"]["similarity_threshold"],
            "automap_max_pairs": CFG["automap"]["max_pairs"],
            "cic_slice_frac": args.cic_slice_frac,
            "cic_slice_seed": args.cic_slice_seed
        }, f, indent=2)

    # NF in-domain (clean + adversarial)
    run_in_domain_nf(nf_df, nf_feats, nf_bin_col, nf_mc_col, outdir)

    # CIC native multiclass
    run_in_domain_cic_native_mc(cic_tr_df, cic_tr_feats, cic_tr_mc_col,
                                cic_te_df, cic_te_feats, cic_te_mc_col, outdir)

    # CIC collapsed-binary (guarded)
    run_in_domain_cic_collapsed(cic_tr_df, cic_tr_feats, cic_tr_mc_col,
                                cic_te_df, cic_te_feats, cic_te_mc_col, outdir, calib_method=args.cic_calib)

    # CIC tiny-slice (ENABLED by default)
    if not args.no_cic_tiny_slice:
        run_cic_with_tiny_benign_slice(
            cic_tr_df, cic_tr_feats, cic_tr_bin_col, cic_tr_mc_col,
            cic_te_df, cic_te_feats, cic_te_bin_col, cic_te_mc_col,
            outdir, slice_frac=args.cic_slice_frac, seed=args.cic_slice_seed
        )

    # Cross-domain (guarded by automap_min)
    run_cross_domain(nf_df, nf_feats, nf_bin_col, nf_mc_col,
                     cic_tr_df, cic_tr_feats, cic_tr_bin_col, cic_tr_mc_col,
                     cic_te_df, cic_te_feats, cic_te_bin_col, cic_te_mc_col,
                     outdir, automap_min=args.automap_min)

if __name__ == "__main__":
    main()

/tmp/ipython-input-3953389373.py:47: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  plt.legend(); plt.tight_layout(); plt.savefig(out_png, dpi=180); plt.close()


[OK] CIC tiny-benign-slice run complete.
